[EduBase](http://www.education.gov.uk/edubase/home.xhtml) allows you to download data for all establishments (which appears to include both closed establishments, open state-funded schools, or open academies and free schools. I will download data for all open state-funded schools. 

In [74]:
## All open state-funded schools
dat = pd.read_csv('../Edubase_datasets/edubaseallstatefunded20170218.csv', low_memory=False)
# dat_links = pd.read_csv('../Edubase_datasets/links_edubaseallstatefunded20170218.csv', low_memory=False)
print len(dat)
print len(dat_links)
print

print 'Establishment status catgories:', dat['EstablishmentStatus (name)'].unique()
print 'Number schools types:', len(dat['TypeOfEstablishment (name)'].unique())
print 
for school_type in sorted(list(dat['TypeOfEstablishment (name)'].unique())):
    print school_type
print 



21917
9586

Establishment status catgories: ['Open' 'Open, but proposed to close']
Number schools types: 22

Academy 16-19 Converter
Academy 16-19 Sponsor Led
Academy Alternative Provision Converter
Academy Alternative Provision Sponsor Led
Academy Converter
Academy Special Converter
Academy Special Sponsor Led
Academy Sponsor Led
Community School
Community Special School
Foundation School
Foundation Special School
Free Schools
Free Schools - 16-19
Free Schools - Alternative Provision
Free Schools Special
LA Nursery School
Pupil Referral Unit
Studio Schools
University Technical College
Voluntary Aided School
Voluntary Controlled School



## School types in EduBase

**Academies:** Academies are publicly funded independent schools. Academies don’t have to follow the national curriculum and can set their own term times. They still have to follow the same rules on admissions, special educational needs and exclusions as other state schools. Academies get money direct from the government, not the local council. They’re run by an academy trust which employs the staff. Some academies have sponsors such as businesses, universities, other schools, faith groups or voluntary groups. Sponsors are responsible for improving the performance of their schools.
<ul>
<li>**Academy Converter:** All schools that have chosen through Governing Body Resolution and application to the Secretary of State to become an Academy under the Academies Act 2010.
<li>**Academy Special:** Special Schools that have chosen through Governing Body Resolution and application to the Secretary of State to become an Academy under the Academies Act 2010. These will be handled differently to Academy convertors and will follow a different process which is currently being developed.
<li>**Academy Sponsor Led:** Academies are all-ability, state-funded schools established and managed by sponsors from a wide range of backgrounds, including high performing schools and colleges, universities, individual philanthropists, businesses, the voluntary sector, and the faith communities.
</ul>

**Community School:** The Local Authority (LA) employs the school's staff, owns the school's land and buildings and is the admissions authority (it has primary responsibility for deciding the arrangements for admitting pupils).

**Community Special School:** A Community special school is the special school equivalent of mainstream Community schools yet are catered wholly or mainly for children with statutory statements of special educational needs.

**Foundation School:** In Foundation schools the governing body is the employer and the admissions authority. The school's land and buildings are either owned by the governing body or by a charitable foundation.

**Foundation Special School:** A Foundation Special schools is a special school equivalent of the mainstream Foundation school catering wholly or mainly for children with statutory statements of special educational needs.

**Free Schools:** Free schools are funded by the government but aren’t run by the local council. They have more control over how they do things. They’re ‘all-ability’ schools, so can’t use academic selection processes like a grammar school. Free schools can, set their own pay and conditions for staff, change the length of school terms and the school day and they don’t have to follow the national curriculum.

**Local Authority (LA) Nursery School:** An LA Nursery school is maintained by a local authority and provides education for children who have attained the age of 2 but are under compulsory school age.



I first read in the edubase csv I downloaded. I then drop all columns except URN and URL, drop observations with empy or invalid urls, and convert the URN column to a number. I then write to csv.

In [60]:
import pandas as pd
# dat = pd.read_csv('../edubasealldata.csv', low_memory=False)
dat = dat[['URN', 'SchoolWebsite']]
dat = dat[dat.SchoolWebsite.notnull()]

# Add 'http:// to start of address where missing (otherwise get error in scraper)
print sum(dat['SchoolWebsite'].str[:4]=='http') 
print sum(dat['SchoolWebsite'].str[:7]=='http://') + sum(dat['SchoolWebsite'].str[:8]=='https://')  # not perfect but close
dat['url_start'] = dat['SchoolWebsite'].str[:4]=='http' 
dat['url_start'].replace(False, 'http://', inplace=True)  #won't work if https
dat['url_start'].replace(True, '', inplace=True)
dat['URL'] = dat['url_start'] + dat['SchoolWebsite']
dat = dat[['URN', 'URL']]

# Fix a couple of bugs in the URLS that are breaking the scraper
old_url_1 = [x for x in dat.loc[dat['URL'].str[:5]=='http;','URL']][0]    ##must be an easier way to get string!!!
old_url_2 = [x for x in dat.loc[dat['URL'].str[:6]=='http:w','URL']][0]
dat.loc[dat['URL'].str[:5]=='http;','URL'] = 'http://' + old_url_1[7:]
dat.loc[dat['URL'].str[:6]=='http:w','URL'] = 'http://' + old_url_2[5:]

# Count unique URNs (should equal numer obs) and URLs (schools could share a URL, so may be less than number obs)
print 'Unique URL:', len(dat.URL.value_counts())
print 'Unique URN:', len(dat.URN.value_counts())
print 'Number obs:', len(dat)

dat.to_csv('/Users/sambarrows/Dropbox/School_websites/Scraping/csvurls.csv', index=False, header=False)

11886
11884
Unique URL: 24516
Unique URN: 26641
Number obs: 26641
